# 🎀 [Day 22](https://adventofcode.com/2019/day/22)

In [1]:
def deal(cards):
    n = len(cards)
    for i in range(n // 2):
        cards[i], cards[n - 1 - i] = cards[n - 1 - i], cards[i]

def cut(cards, N):
    if N < 0:
        N = len(cards) + N
    cut = cards[:N]
    for i in range(len(cards) - N):
        cards[i] = cards[N + i]
    for i in range(N):
        cards[len(cards) - N + i] = cut[i]

def deal_with_increment(cards, N):
    n = len(cards)
    deck = [0] * n
    index = 0
    for i in range(n):
        deck[index] = cards[i]
        index = (index + N) % n
    return deck


def shuffle(commands, n=10007):
    """Apply shuffle instructions"""
    cards = list(range(n))
    for line in commands:
        if line.startswith('cut'):
            N = int(line.split()[-1])
            cut(cards, N)
        elif line.startswith('deal with increment'):
            N = int(line.split()[-1])
            cards = deal_with_increment(cards, N)
        elif line.startswith('deal into'):
            deal(cards)
    return cards
        
    
def track_card(commands, n=10007, card=2019):
    """Position of a card is actually independent from the others"""
    index = card
    for line in commands:
        if line.startswith('cut'):
            N = int(line.split()[-1])
            N+= n if N < 0 else 0
            if index < N:
                index = n - N + index
            else:
                index -= N
        elif line.startswith('deal with increment'):
            N = int(line.split()[-1])
            index = (N * index) % n
        elif line.startswith('deal into'):
            index = n - 1 - index
    return index

In [2]:
with open('inputs/day22.txt', 'r') as f:
    inputs = f.read().splitlines()
    
print("After one suffle, the 2019-th card is at position", shuffle(inputs).index(2019))
print("Verifying `track_card` implementation:", track_card(inputs, card=2019))

After one suffle, the 2019-th card is at position 6431
Verifying `track_card` implementation: 6431


**Part 2** must be solved with modular arithmetic. Building off `track_cards`, the three deck shuffling operations can be writen as:
  * `deal(x)` = -x + n - 1
  * `cut(x, N)` = x - N + n  [n]
  * `deal_with_increment(x, N)` = x * N [n]
  
 And if we invert them:
  * `inv_deal(x)` =  `deal(x)`
  * `inv_cut(x, N)` = `cut(x, n - N)` 
  * `deal_with_increment(x, N)` = x * N [n]
  
 Thus shuffling is a sequence of linear operations (modulo n), hence can be rewritten as a linear operation `shuffle(x)` = Ax + B.
 
 Thus iterating shuffle gives us:
   * `shuffle(x)`$^2$ = A$^2$ x + AB + B
   * `shuffle(x)`$^3$ = A$^3$ x + A$^2$B + AB + B
   *...
   * `shuffle(x)`$^k$ = A$^k$ x + A$^{k - 1}$B + ... AB + B
   
The later terms are the sum of a geometric progression hence can be written: `shuffle(x)`$^k$ = A$^k$ x + $\frac{1 - \text{A}^k}{1 - \text{A}}$ B

In [3]:
def get_coefficients(commands, n, inv=False):
    """Position of a card is actually independent from the others"""
    A = 1
    B = 0
    for line in (commands[::-1] if inv else commands):
        if line.startswith('cut'):
            N = int(line.split()[-1])
            if inv:
                B += N
            else:
                B += n - N
        elif line.startswith('deal with increment'):
            N = int(line.split()[-1])
            if inv:
                invmod = pow(N, -1, n)
                B *= invmod
                A *= invmod
            else:
                B *= N
                A *= N
        elif line.startswith('deal into'):
            B = - B + n - 1
            A = - A
    return A, B


def repeat(x, k, A, B, n):
    Ak = pow(A, k, n) # modulo
    out = Ak * x % n
    invmod = pow(1 - A, -1, n)
    out += (B % n) * ((1 - Ak) % n) * invmod
    return int(out % n)

In [4]:
print("\033[1mTest repeat formula on Part 1 inputs\033[0m")
card = 2019
n = 10007
A, B = get_coefficients(inputs, n)
k = 5
index = card
for _ in range(k):
    index = track_card(inputs, card=index)
print("Brute force says", index)
print("Modular arithmetics says", repeat(card, k, A, B, n))

Test repeat formula on Part 1 inputs
Brute force says 4976
Modular arithmetics says 4976


In [5]:
print("\033[1mTest inversion formula\033[0m")
card = 2020
n = 10007
A, B = get_coefficients(inputs, n, inv=True)
cards = shuffle(inputs, n)
print("Brute force says", cards[card])
print("Modular arithemtics says", (A * card + B) % n)

Test inversion formula
Brute force says 9632
Modular arithemtics says 9632


In [6]:
card = 2020
n = 119315717514047
k = 101741582076661
A, B = get_coefficients(inputs, n, inv=True)
print("The solution to part 2 is", repeat(card, k, A, B, n))

The solution to part 2 is 100982886178958
